In [1]:
import sys, os
from os import path
sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
# sys.path.append('/root/bdrisk/risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *

In [59]:
sql = f"""\
SELECT * FROM {TableList.BD_RISK_DETAIL_WAREHOUSE_WH1_C.value} WHERE ISCURRENT = 1 AND ORG_CODE LIKE '91310115688774070B'
"""
CUS_DATA = Read_Oracle().read_oracle(sql=sql, database='dbods')

2021-07-01 11:22:27.632 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 5.803s


In [60]:
sql = f"""\
SELECT * FROM {TableList.BD_RISK_DETAIL_WAREHOUSE_WH1_S.value} WHERE ISCURRENT = 1 AND ORG_CODE LIKE '91310115688774070B'
"""
STOCK_DATA = Read_Oracle().read_oracle(sql=sql, database='dbods')

2021-07-01 11:22:29.819 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 2.046s


In [61]:
entry_list = CUS_DATA[['ENTRY_NO','GOODS_NO']].copy()
entry_list = entry_list.drop_duplicates()
entry_list['conn_flag'] = 1

In [62]:
entry_only = CUS_DATA[['ENTRY_NO']].copy()
entry_only = entry_only.drop_duplicates()
entry_only['conn_flag_e'] = 1

In [74]:
df = pd.merge(STOCK_DATA,entry_list,how = 'left',left_on=['ENTRY_NO','ENTRY_GDS_SEQNO'],right_on=['ENTRY_NO','GOODS_NO'])
df = pd.merge(df,entry_only,how = 'left',left_on=['ENTRY_NO'],right_on=['ENTRY_NO'])

In [91]:
df_match = df[-pd.isnull(df['conn_flag_e'])].copy()
df_result = df[pd.isnull(df['conn_flag_e'])].copy()
df_result = df_result.drop(columns = ['ID','ISCURRENT','CHECK_TIME','conn_flag_e','conn_flag','LASTUPDATE'])
df_match = df_match.drop(columns = ['ID','ISCURRENT','CHECK_TIME','conn_flag_e','LASTUPDATE'])
df_result['RISK_LABEL'] = '未匹配到相应报关单'
df_result['SCORE'] = 0

In [23]:
df_match

,ORG_CODE,CUSTOMS_CODE,STOCK_BILL_TYPE,STOCK_TYPE,BILL_TYPE,BUSINESS_TYPE,BILL_STATUS,CLASSIFY_TYPE,SUPV_MODE,I_E_PORT,...,G_UNIT,QTY_1,UNIT_1,QTY_2,UNIT_2,TRADE_CURR,TRADE_TOTAL,STARTDT,GOODS_NO,conn_flag
12011,91310115688774070B,2248,2,1,1,3,nan,I,nan,2248,...,001,706.000,001,2864.155,035,502,68426.48,2020-05-20 15:17:14,1.0,1.0
12012,91310115688774070B,2248,2,1,1,3,nan,I,nan,2248,...,001,34.908,035,0.000,nan,502,10139.73,2020-05-20 15:17:14,10.0,1.0
12013,91310115688774070B,2248,2,1,1,3,nan,I,nan,2248,...,007,998.364,035,33.000,001,502,7728.24,2020-05-20 15:17:14,11.0,1.0
12014,91310115688774070B,2248,2,1,1,3,nan,I,nan,2248,...,001,7.000,001,25.600,035,502,5652.62,2020-05-20 15:17:14,12.0,1.0
12015,91310115688774070B,2248,2,1,1,3,nan,I,nan,2248,...,007,171.000,035,175.000,001,502,2446.90,2020-05-20 15:17:14,13.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25768,91310115688774070B,2233,1,1,1,3,nan,I,nan,2233,...,007,4.950,035,0.000,nan,142,1274.19,2020-06-03 15:27:10,4.0,1.0
25769,91310115688774070B,2233,1,1,1,3,nan,I,nan,2233,...,007,7.800,035,0.000,nan,142,300.20,2020-06-03 15:27:10,5.0,1.0
25770,91310115688774070B,2248,1,1,1,3,nan,I,nan,2248,...,001,9.000,001,8154.000,035,142,676180.95,2020-06-08 09:20:06,1.0,1.0
25771,91310115688774070B,2248,1,1,1,3,nan,I,nan,2248,...,001,2.000,001,2130.000,035,142,215664.22,2020-06-08 09:20:06,2.0,1.0


In [90]:
def thr_sup(num_x):
    if pd.isnull(num_x):
        return 0
    else:
        return (1+0.05)*num_x
    
def thr_inf(num_x):
    if pd.isnull(num_x):
        return 0
    else:
        return (1-0.05)*num_x


def str_in(row):
    a = row['G_NAME']
    b = row['GOODS_NAME']
    if a in b:
        return True
    elif b in a:
        return True
    else:
        return False
    
def str_in_2(row):
    a = row['G_NAME_m']
    b = row['GOODS_NAME']
    if a in b:
        return True
    elif b in a:
        return True
    else:
        return False

In [92]:
x = df_match[pd.isnull(df_match['conn_flag'])].copy()
x = x.drop(columns=['conn_flag'])
x['RISK_LABEL'] = '未能匹配到相应货物'
x['SCORE'] = 0
df_result = pd.concat([df_result,x])
df_match = df_match[-pd.isnull(df_match['conn_flag'])].copy()
df_match = df_match.drop(columns=['conn_flag'])

In [85]:
df_match_c = df_match.copy()

In [96]:
df_match = df_match_c.copy()

In [93]:
df_match.shape

(5850, 27)

In [95]:
df_result.shape

(19942, 29)

(5850, 28)
(5820, 28)
(30, 28)
(5821, 29)
(29, 29)
(5821, 29)
(29, 29)


In [97]:
if df_match.shape[0]!=0:
    
    CHECK_DATA = CUS_DATA[['ENTRY_NO','GOODS_NO','GOODS_NAME']].copy()
    df_match = pd.merge(df_match,CHECK_DATA,how='inner',left_on=['ENTRY_NO','GOODS_NO'],right_on=['ENTRY_NO','GOODS_NO'])

    test = df_match[df_match['G_NAME']!=df_match['GOODS_NAME']].copy()
    df_str_fine = df_match[df_match['G_NAME']==df_match['GOODS_NAME']].copy()
    test2 = test[-test.apply(str_in,axis=1)].copy()
    test = test[test.apply(str_in,axis=1)].copy()
    df_str_fine = pd.concat([df_str_fine,test])

    test2['GOODS_NAME'] = test2['GOODS_NAME'].apply(lambda x : x.replace("（","("))
    test2['GOODS_NAME'] = test2['GOODS_NAME'].apply(lambda x : x.replace("）",")"))
    test2['G_NAME_m'] = test2['G_NAME'].apply(lambda x : x.replace("（","("))
    test2['G_NAME_m'] = test2['G_NAME_m'].apply(lambda x : x.replace("）",")"))
    test = test2[test2['GOODS_NAME'] == test2['G_NAME_m']].copy()
    test2 = test2[test2['GOODS_NAME'] != test2['G_NAME_m']].copy()
    df_str_fine = pd.concat([df_str_fine,test])

    test = test2[test2.apply(str_in_2,axis=1)].copy()
    test2 = test2[-test2.apply(str_in_2,axis=1)].copy()
    df_str_fine = pd.concat([df_str_fine,test])

    df_str_fine['str_label'] = '品名可匹配'
    df_str_fine['str_score'] = 0
    test2['str_label'] = '品名不匹配'
    test2['str_score'] = 0

    df_match = pd.concat([df_str_fine,test2])

    
    CHECK_DATA = CUS_DATA[['ENTRY_NO','GOODS_NO','DECL_TOTAL']].copy()
    CHECK_DATA['trade_thr_sup'] = CHECK_DATA['DECL_TOTAL'].apply(thr_sup)
    CHECK_DATA['trade_thr_inf'] = CHECK_DATA['DECL_TOTAL'].apply(thr_inf)
    CHECK_DATA = CHECK_DATA.drop(columns=['DECL_TOTAL'])
    df_match = pd.merge(df_match,CHECK_DATA,how='inner',left_on=['ENTRY_NO','GOODS_NO'],right_on=['ENTRY_NO','GOODS_NO'])
    x = df_match[(df_match['TRADE_TOTAL']>df_match['trade_thr_sup'])|(df_match['TRADE_TOTAL']<df_match['trade_thr_inf'])].copy()
    match_count = df_match.shape[0]
    x = x.drop(columns=['trade_thr_sup','trade_thr_inf'])
    x['RISK_LABEL'] = '贸易金额不符'
    x['SCORE'] = -1/match_count*100
    df_match = df_match[(df_match['TRADE_TOTAL']<=df_match['trade_thr_sup'])&(df_match['TRADE_TOTAL']>=df_match['trade_thr_inf'])].copy()
    df_match = df_match.drop(columns=['trade_thr_sup','trade_thr_inf'])
    df_result = pd.concat([df_result,x])

In [98]:
if df_match.shape[0]!=0:
    CHECK_DATA = CUS_DATA[['ENTRY_NO','GOODS_NO','G_UNIT','UNIT_1','UNIT_2','G_QTY','QTY_1','QTY_2']].copy()
    CHECK_DATA['qty_thr_sup_0'] = CHECK_DATA['G_QTY'].apply(thr_sup)
    CHECK_DATA['qty_thr_inf_0'] = CHECK_DATA['G_QTY'].apply(thr_inf)
    CHECK_DATA['qty_thr_sup_1'] = CHECK_DATA['QTY_1'].apply(thr_sup)
    CHECK_DATA['qty_thr_inf_1'] = CHECK_DATA['QTY_1'].apply(thr_inf)
    CHECK_DATA['qty_thr_sup_2'] = CHECK_DATA['QTY_2'].apply(thr_sup)
    CHECK_DATA['qty_thr_inf_2'] = CHECK_DATA['QTY_2'].apply(thr_inf)
    CHECK_DATA = CHECK_DATA.drop(columns=['G_QTY','QTY_1','QTY_2'])
    CHECK_DATA.columns = ['ENTRY_NO','GOODS_NO','UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2']
    CHECK_DATA['UNIT_CHECK_0'] = CHECK_DATA['UNIT_CHECK_0'].astype(str)
    CHECK_DATA['UNIT_CHECK_1'] = CHECK_DATA['UNIT_CHECK_1'].astype(str)
    CHECK_DATA['UNIT_CHECK_2'] = CHECK_DATA['UNIT_CHECK_2'].astype(str)

    df_match = pd.merge(df_match,CHECK_DATA,how='inner',left_on=['ENTRY_NO','GOODS_NO'],right_on=['ENTRY_NO','GOODS_NO'])

    x = df_match[df_match['G_UNIT']==df_match['UNIT_CHECK_0']].copy()
    df_match = df_match[df_match['G_UNIT']!=df_match['UNIT_CHECK_0']].copy()

    y = x[(x['QTY']>x['qty_thr_sup_0'])|(x['QTY']<x['qty_thr_inf_0'])].copy()
    x = x[(x['QTY']<=x['qty_thr_sup_0'])&(x['QTY']>=x['qty_thr_inf_0'])].copy()
    y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    y['RISK_LABEL'] = '申报数量异常'
    y['SCORE'] = -1/match_count*100
    x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    x['RISK_LABEL'] = '数据正常'
    x['SCORE'] = 0
    df_result = pd.concat([df_result,x,y])

if df_match.shape[0]!=0:
    x = df_match[df_match['G_UNIT']==df_match['UNIT_CHECK_1']].copy()
    df_match = df_match[df_match['G_UNIT']!=df_match['UNIT_CHECK_1']].copy()

    y = x[(x['QTY']>x['qty_thr_sup_1'])|(x['QTY']<x['qty_thr_inf_1'])].copy()
    x = x[(x['QTY']<=x['qty_thr_sup_1'])&(x['QTY']>=x['qty_thr_inf_1'])].copy()
    y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    y['RISK_LABEL'] = '申报数量异常'
    y['SCORE'] = -1/match_count*100
    x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    x['RISK_LABEL'] = '数据正常'
    x['SCORE'] = 0
    df_result = pd.concat([df_result,x,y])

if df_match.shape[0]!=0:
    x = df_match[df_match['G_UNIT']==df_match['UNIT_CHECK_2']].copy()
    df_match = df_match[df_match['G_UNIT']!=df_match['UNIT_CHECK_2']].copy()

    y = x[(x['QTY']>x['qty_thr_sup_2'])|(x['QTY']<x['qty_thr_inf_2'])].copy()
    x = x[(x['QTY']<=x['qty_thr_sup_2'])&(x['QTY']>=x['qty_thr_inf_2'])].copy()
    y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    y['RISK_LABEL'] = '申报数量异常'
    y['SCORE'] = -1/match_count*100
    x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    x['RISK_LABEL'] = '数据正常'
    x['SCORE'] = 0
    df_result = pd.concat([df_result,x,y])

In [99]:
if df_match.shape[0]!=0:
    x = df_match[df_match['UNIT_1']==df_match['UNIT_CHECK_0']].copy()
    df_match = df_match[df_match['UNIT_1']!=df_match['UNIT_CHECK_0']].copy()

    y = x[(x['QTY_1']>x['qty_thr_sup_0'])|(x['QTY_1']<x['qty_thr_inf_0'])].copy()
    x = x[(x['QTY_1']<=x['qty_thr_sup_0'])&(x['QTY_1']>=x['qty_thr_inf_0'])].copy()
    y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    y['RISK_LABEL'] = '申报数量无法匹配，法定数量1异常'
    y['SCORE'] = -1/match_count*100
    x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    x['RISK_LABEL'] = '数据正常'
    x['SCORE'] = 0
    df_result = pd.concat([df_result,x,y])

if df_match.shape[0]!=0:
    x = df_match[df_match['UNIT_1']==df_match['UNIT_CHECK_1']].copy()
    df_match = df_match[df_match['UNIT_1']!=df_match['UNIT_CHECK_1']].copy()

    y = x[(x['QTY_1']>x['qty_thr_sup_1'])|(x['QTY_1']<x['qty_thr_inf_1'])].copy()
    x = x[(x['QTY_1']<=x['qty_thr_sup_1'])&(x['QTY_1']>=x['qty_thr_inf_1'])].copy()
    y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    y['RISK_LABEL'] = '申报数量无法匹配，法定数量1异常'
    y['SCORE'] = -1/match_count*100
    x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    x['RISK_LABEL'] = '数据正常'
    x['SCORE'] = 0
    df_result = pd.concat([df_result,x,y])

if df_match.shape[0]!=0:
    x = df_match[df_match['UNIT_1']==df_match['UNIT_CHECK_2']].copy()
    df_match = df_match[df_match['UNIT_1']!=df_match['UNIT_CHECK_2']].copy()

    y = x[(x['QTY_1']>x['qty_thr_sup_2'])|(x['QTY_1']<x['qty_thr_inf_2'])].copy()
    x = x[(x['QTY_1']<=x['qty_thr_sup_2'])&(x['QTY_1']>=x['qty_thr_inf_2'])].copy()
    y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    y['RISK_LABEL'] = '申报数量无法匹配，法定数量1异常'
    y['SCORE'] = -1/match_count*100
    x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    x['RISK_LABEL'] = '数据正常'
    x['SCORE'] = 0
    df_result = pd.concat([df_result,x,y])

In [100]:
if df_match.shape[0]!=0:
    x = df_match[df_match['UNIT_2']==df_match['UNIT_CHECK_0']].copy()
    df_match = df_match[df_match['UNIT_2']!=df_match['UNIT_CHECK_0']].copy()

    y = x[(x['QTY_2']>x['qty_thr_sup_0'])|(x['QTY_2']<x['qty_thr_inf_0'])].copy()
    x = x[(x['QTY_2']<=x['qty_thr_sup_0'])&(x['QTY_2']>=x['qty_thr_inf_0'])].copy()
    y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    y['RISK_LABEL'] = '申报数量与法定数量1均无法匹配，法定数量2异常'
    y['SCORE'] = -1/match_count*100
    x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    x['RISK_LABEL'] = '数据正常'
    x['SCORE'] = 0
    df_result = pd.concat([df_result,x,y])

if df_match.shape[0]!=0:
    x = df_match[df_match['UNIT_2']==df_match['UNIT_CHECK_1']].copy()
    df_match = df_match[df_match['UNIT_2']!=df_match['UNIT_CHECK_1']].copy()

    y = x[(x['QTY_2']>x['qty_thr_sup_1'])|(x['QTY_2']<x['qty_thr_inf_1'])].copy()
    x = x[(x['QTY_2']<=x['qty_thr_sup_1'])&(x['QTY_2']>=x['qty_thr_inf_1'])].copy()
    y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    y['RISK_LABEL'] = '申报数量与法定数量1均无法匹配，法定数量2异常'
    y['SCORE'] = -1/match_count*100
    x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    x['RISK_LABEL'] = '数据正常'
    x['SCORE'] = 0
    df_result = pd.concat([df_result,x,y])

if df_match.shape[0]!=0:
    x = df_match[df_match['UNIT_2']==df_match['UNIT_CHECK_2']].copy()
    z = df_match[df_match['UNIT_2']!=df_match['UNIT_CHECK_2']].copy()

    y = x[(x['QTY_2']>x['qty_thr_sup_2'])|(x['QTY_2']<x['qty_thr_inf_2'])].copy()
    x = x[(x['QTY_2']<=x['qty_thr_sup_2'])&(x['QTY_2']>=x['qty_thr_inf_2'])].copy()
    y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    y['RISK_LABEL'] = '申报数量与法定数量1均无法匹配，法定数量2异常'
    y['SCORE'] = -1/match_count*100
    x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    x['RISK_LABEL'] = '数据正常'
    x['SCORE'] = 0
    z = z.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
    z['RISK_LABEL'] = '申报数量、法定数量1与法定数量2均无法匹配'
    z['SCORE'] = -1/match_count*100
    df_result = pd.concat([df_result,x,y,z])

In [103]:
df_result.columns

Index(['ORG_CODE', 'CUSTOMS_CODE', 'STOCK_BILL_TYPE', 'STOCK_TYPE',
       'BILL_TYPE', 'BUSINESS_TYPE', 'BILL_STATUS', 'CLASSIFY_TYPE',
       'SUPV_MODE', 'I_E_PORT', 'TRAF_MODE', 'ENTRY_NO', 'ENTRY_GDS_SEQNO',
       'HS_CODE', 'G_NAME', 'G_MODEL', 'ORIGIN_COUNTRY_CODE', 'QTY', 'G_UNIT',
       'QTY_1', 'UNIT_1', 'QTY_2', 'UNIT_2', 'TRADE_CURR', 'TRADE_TOTAL',
       'STARTDT', 'GOODS_NO', 'RISK_LABEL', 'SCORE', 'GOODS_NAME', 'G_NAME_m',
       'str_label', 'str_score'],
      dtype='object')

In [105]:
test = df_result[['RISK_LABEL','SCORE','str_label','str_score']].copy()

In [107]:
test_c = test.copy()

In [108]:
test = test[-pd.isnull(test['str_label'])].copy()

In [110]:
def label_merge(row):
    str_a = row['RISK_LABEL']
    str_b = row['str_label']
    return f'{str_a}, {str_b}'

In [111]:
test['test'] = test.apply(label_merge,axis=1)

In [112]:
test

,RISK_LABEL,SCORE,str_label,str_score,test
3399,贸易金额不符,-0.017094,品名可匹配,0.0,"贸易金额不符, 品名可匹配"
3920,贸易金额不符,-0.017094,品名可匹配,0.0,"贸易金额不符, 品名可匹配"
3929,贸易金额不符,-0.017094,品名可匹配,0.0,"贸易金额不符, 品名可匹配"
3933,贸易金额不符,-0.017094,品名可匹配,0.0,"贸易金额不符, 品名可匹配"
4290,贸易金额不符,-0.017094,品名可匹配,0.0,"贸易金额不符, 品名可匹配"
...,...,...,...,...,...
5748,申报数量异常,-0.017094,品名不匹配,0.0,"申报数量异常, 品名不匹配"
5749,申报数量无法匹配，法定数量1异常,-0.017094,品名不匹配,0.0,"申报数量无法匹配，法定数量1异常, 品名不匹配"
5750,申报数量无法匹配，法定数量1异常,-0.017094,品名不匹配,0.0,"申报数量无法匹配，法定数量1异常, 品名不匹配"
5751,申报数量无法匹配，法定数量1异常,-0.017094,品名不匹配,0.0,"申报数量无法匹配，法定数量1异常, 品名不匹配"


In [102]:
df_result.shape

(25792, 33)

In [56]:
STOCK_DATA.shape

(25792, 30)

In [165]:
df_result = df_result.drop(columns=['GOODS_NO'])

In [167]:
df_result = df_result.reset_index().rename(columns = {'index':'ID'})
now = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
df_result['CHECK_TIME']=now
df_result['CHECK_TIME']= pd.to_datetime(df_result['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')

df_result['SCORE'] = df_result['SCORE'].astype(float)

In [168]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25792 entries, 0 to 25791
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   25792 non-null  int64         
 1   ORG_CODE             25792 non-null  object        
 2   CUSTOMS_CODE         25792 non-null  object        
 3   STOCK_BILL_TYPE      25792 non-null  object        
 4   STOCK_TYPE           25792 non-null  object        
 5   BILL_TYPE            25792 non-null  object        
 6   BUSINESS_TYPE        25792 non-null  object        
 7   BILL_STATUS          25792 non-null  object        
 8   CLASSIFY_TYPE        25792 non-null  object        
 9   SUPV_MODE            25792 non-null  object        
 10  I_E_PORT             25792 non-null  object        
 11  TRAF_MODE            25792 non-null  object        
 12  ENTRY_NO             25792 non-null  object        
 13  ENTRY_GDS_SEQNO      25792 non-

In [102]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25792 entries, 0 to 3
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ORG_CODE             25792 non-null  object        
 1   CUSTOMS_CODE         25792 non-null  object        
 2   STOCK_BILL_TYPE      25792 non-null  object        
 3   STOCK_TYPE           25792 non-null  object        
 4   BILL_TYPE            25792 non-null  object        
 5   BUSINESS_TYPE        25792 non-null  object        
 6   BILL_STATUS          25792 non-null  object        
 7   CLASSIFY_TYPE        25792 non-null  object        
 8   SUPV_MODE            25792 non-null  object        
 9   I_E_PORT             25792 non-null  object        
 10  TRAF_MODE            25792 non-null  object        
 11  ENTRY_NO             25792 non-null  object        
 12  ENTRY_GDS_SEQNO      25792 non-null  int64         
 13  HS_CODE              25792 non-null

In [5]:
import sys, os
from os import path
# sys.path.append('/root/bdrisk/risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))

sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *


class WarehouseModelWh1(object):
    def __init__(self, org_code, params, base_time, child_task_id):
        self.model_code = 'WAREHOUSE'
        self.child_model_code = 'WH1'
        self.child_task_id = child_task_id
        Risk_logger(child_task_id=self.child_task_id, exec_status=None)

        # 参数读取
        self.org_code = org_code
        # [正常，匹配不上报关单，匹配不上货物，贸易金额不符，法定数量不符，申报数量1不符，申报数量2不符，三种数量无法对应]
        self.score_crit = [0,0,0,-1,-1,-1,-1,-1]


    def model_wh1(self):

        def thr_sup(num_x):
            if pd.isnull(num_x):
                return 0
            else:
                return (1+0.05)*num_x
            
        def thr_inf(num_x):
            if pd.isnull(num_x):
                return 0
            else:
                return (1-0.05)*num_x

        sql = f"""\
        SELECT * FROM {TableList.BD_RISK_DETAIL_WAREHOUSE_WH1_C.value} WHERE ISCURRENT = 1 AND ORG_CODE LIKE '91310115688774070B'
        """
        CUS_DATA = Read_Oracle().read_oracle(sql=sql, database='dbods')

        sql = f"""\
        SELECT * FROM {TableList.BD_RISK_DETAIL_WAREHOUSE_WH1_S.value} WHERE ISCURRENT = 1 AND ORG_CODE LIKE '91310115688774070B'
        """
        STOCK_DATA = Read_Oracle().read_oracle(sql=sql, database='dbods')

        entry_list = CUS_DATA[['ENTRY_NO','GOODS_NO']].copy()
        entry_list = entry_list.drop_duplicates()
        entry_list['conn_flag'] = 1

        entry_only = CUS_DATA[['ENTRY_NO']].copy()
        entry_only = entry_only.drop_duplicates()
        entry_only['conn_flag_e'] = 1

        df = pd.merge(STOCK_DATA,entry_list,how = 'left',left_on=['ENTRY_NO','ENTRY_GDS_SEQNO'],right_on=['ENTRY_NO','GOODS_NO'])
        df = pd.merge(df,entry_only,how = 'left',left_on=['ENTRY_NO'],right_on=['ENTRY_NO'])

        df_match = df[-pd.isnull(df['conn_flag_e'])].copy()
        df_result = df[pd.isnull(df['conn_flag_e'])].copy()
        df_result = df_result.drop(columns = ['ID','ISCURRENT','CHECK_TIME','conn_flag_e','conn_flag','LASTUPDATE'])
        df_match = df_match.drop(columns = ['ID','ISCURRENT','CHECK_TIME','conn_flag_e','LASTUPDATE'])
        df_result['RISK_LABEL'] = '未匹配到相应报关单'
        df_result['SCORE'] = self.score_crit[1]

        x = df_match[pd.isnull(df_match['conn_flag'])].copy()
        x = x.drop(columns=['conn_flag'])
        x['RISK_LABEL'] = '未能匹配到相应货物'
        x['SCORE'] = self.score_crit[2]
        df_result = pd.concat([df_result,x])
        df_match = df_match[-pd.isnull(df_match['conn_flag'])].copy()
        df_match = df_match.drop(columns=['conn_flag'])

        if df_match.shape[0]!=0:
            CHECK_DATA = CUS_DATA[['ENTRY_NO','GOODS_NO','DECL_TOTAL']].copy()
            CHECK_DATA['trade_thr_sup'] = CHECK_DATA['DECL_TOTAL'].apply(thr_sup)
            CHECK_DATA['trade_thr_inf'] = CHECK_DATA['DECL_TOTAL'].apply(thr_inf)
            CHECK_DATA = CHECK_DATA.drop(columns=['DECL_TOTAL'])
            df_match = pd.merge(df_match,CHECK_DATA,how='inner',left_on=['ENTRY_NO','GOODS_NO'],right_on=['ENTRY_NO','GOODS_NO'])
            x = df_match[(df_match['TRADE_TOTAL']>df_match['trade_thr_sup'])|(df_match['TRADE_TOTAL']<df_match['trade_thr_inf'])].copy()
            match_count = df_match.shape[0]
            x = x.drop(columns=['trade_thr_sup','trade_thr_inf'])
            x['RISK_LABEL'] = '贸易金额不符'
            x['SCORE'] = -self.score_crit[3]/match_count*100
            df_match = df_match[(df_match['TRADE_TOTAL']<=df_match['trade_thr_sup'])&(df_match['TRADE_TOTAL']>=df_match['trade_thr_inf'])].copy()
            df_match = df_match.drop(columns=['trade_thr_sup','trade_thr_inf'])
            df_result = pd.concat([df_result,x])

        if df_match.shape[0]!=0:
            CHECK_DATA = CUS_DATA[['ENTRY_NO','GOODS_NO','G_UNIT','UNIT_1','UNIT_2','G_QTY','QTY_1','QTY_2']].copy()
            CHECK_DATA['qty_thr_sup_0'] = CHECK_DATA['G_QTY'].apply(thr_sup)
            CHECK_DATA['qty_thr_inf_0'] = CHECK_DATA['G_QTY'].apply(thr_inf)
            CHECK_DATA['qty_thr_sup_1'] = CHECK_DATA['QTY_1'].apply(thr_sup)
            CHECK_DATA['qty_thr_inf_1'] = CHECK_DATA['QTY_1'].apply(thr_inf)
            CHECK_DATA['qty_thr_sup_2'] = CHECK_DATA['QTY_2'].apply(thr_sup)
            CHECK_DATA['qty_thr_inf_2'] = CHECK_DATA['QTY_2'].apply(thr_inf)
            CHECK_DATA = CHECK_DATA.drop(columns=['G_QTY','QTY_1','QTY_2'])
            CHECK_DATA.columns = ['ENTRY_NO','GOODS_NO','UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2']
            CHECK_DATA['UNIT_CHECK_0'] = CHECK_DATA['UNIT_CHECK_0'].astype(str)
            CHECK_DATA['UNIT_CHECK_1'] = CHECK_DATA['UNIT_CHECK_1'].astype(str)
            CHECK_DATA['UNIT_CHECK_2'] = CHECK_DATA['UNIT_CHECK_2'].astype(str)

            df_match = pd.merge(df_match,CHECK_DATA,how='inner',left_on=['ENTRY_NO','GOODS_NO'],right_on=['ENTRY_NO','GOODS_NO'])

            x = df_match[df_match['G_UNIT']==df_match['UNIT_CHECK_0']].copy()
            df_match = df_match[df_match['G_UNIT']!=df_match['UNIT_CHECK_0']].copy()

            y = x[(x['QTY']>x['qty_thr_sup_0'])|(x['QTY']<x['qty_thr_inf_0'])].copy()
            x = x[(x['QTY']<=x['qty_thr_sup_0'])&(x['QTY']>=x['qty_thr_inf_0'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量异常'
            y['SCORE'] = -self.score_crit[4]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['G_UNIT']==df_match['UNIT_CHECK_1']].copy()
            df_match = df_match[df_match['G_UNIT']!=df_match['UNIT_CHECK_1']].copy()

            y = x[(x['QTY']>x['qty_thr_sup_1'])|(x['QTY']<x['qty_thr_inf_1'])].copy()
            x = x[(x['QTY']<=x['qty_thr_sup_1'])&(x['QTY']>=x['qty_thr_inf_1'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量异常'
            y['SCORE'] = -self.score_crit[4]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['G_UNIT']==df_match['UNIT_CHECK_2']].copy()
            df_match = df_match[df_match['G_UNIT']!=df_match['UNIT_CHECK_2']].copy()

            y = x[(x['QTY']>x['qty_thr_sup_2'])|(x['QTY']<x['qty_thr_inf_2'])].copy()
            x = x[(x['QTY']<=x['qty_thr_sup_2'])&(x['QTY']>=x['qty_thr_inf_2'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量异常'
            y['SCORE'] = -self.score_crit[4]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['UNIT_1']==df_match['UNIT_CHECK_0']].copy()
            df_match = df_match[df_match['UNIT_1']!=df_match['UNIT_CHECK_0']].copy()

            y = x[(x['QTY_1']>x['qty_thr_sup_0'])|(x['QTY_1']<x['qty_thr_inf_0'])].copy()
            x = x[(x['QTY_1']<=x['qty_thr_sup_0'])&(x['QTY_1']>=x['qty_thr_inf_0'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量无法匹配，法定数量1异常'
            y['SCORE'] = -self.score_crit[5]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['UNIT_1']==df_match['UNIT_CHECK_1']].copy()
            df_match = df_match[df_match['UNIT_1']!=df_match['UNIT_CHECK_1']].copy()

            y = x[(x['QTY_1']>x['qty_thr_sup_1'])|(x['QTY_1']<x['qty_thr_inf_1'])].copy()
            x = x[(x['QTY_1']<=x['qty_thr_sup_1'])&(x['QTY_1']>=x['qty_thr_inf_1'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量无法匹配，法定数量1异常'
            y['SCORE'] = -self.score_crit[5]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['UNIT_1']==df_match['UNIT_CHECK_2']].copy()
            df_match = df_match[df_match['UNIT_1']!=df_match['UNIT_CHECK_2']].copy()

            y = x[(x['QTY_1']>x['qty_thr_sup_2'])|(x['QTY_1']<x['qty_thr_inf_2'])].copy()
            x = x[(x['QTY_1']<=x['qty_thr_sup_2'])&(x['QTY_1']>=x['qty_thr_inf_2'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量无法匹配，法定数量1异常'
            y['SCORE'] = -self.score_crit[5]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['UNIT_2']==df_match['UNIT_CHECK_0']].copy()
            df_match = df_match[df_match['UNIT_2']!=df_match['UNIT_CHECK_0']].copy()

            y = x[(x['QTY_2']>x['qty_thr_sup_0'])|(x['QTY_2']<x['qty_thr_inf_0'])].copy()
            x = x[(x['QTY_2']<=x['qty_thr_sup_0'])&(x['QTY_2']>=x['qty_thr_inf_0'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量与法定数量1均无法匹配，法定数量2异常'
            y['SCORE'] = -self.score_crit[6]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['UNIT_2']==df_match['UNIT_CHECK_1']].copy()
            df_match = df_match[df_match['UNIT_2']!=df_match['UNIT_CHECK_1']].copy()

            y = x[(x['QTY_2']>x['qty_thr_sup_1'])|(x['QTY_2']<x['qty_thr_inf_1'])].copy()
            x = x[(x['QTY_2']<=x['qty_thr_sup_1'])&(x['QTY_2']>=x['qty_thr_inf_1'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量与法定数量1均无法匹配，法定数量2异常'
            y['SCORE'] = -self.score_crit[6]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['UNIT_2']==df_match['UNIT_CHECK_2']].copy()
            z = df_match[df_match['UNIT_2']!=df_match['UNIT_CHECK_2']].copy()

            y = x[(x['QTY_2']>x['qty_thr_sup_2'])|(x['QTY_2']<x['qty_thr_inf_2'])].copy()
            x = x[(x['QTY_2']<=x['qty_thr_sup_2'])&(x['QTY_2']>=x['qty_thr_inf_2'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量与法定数量1均无法匹配，法定数量2异常'
            y['SCORE'] = -self.score_crit[6]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            z = z.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            z['RISK_LABEL'] = '申报数量、法定数量1与法定数量2均无法匹配'
            z['SCORE'] = -self.score_crit[7]/match_count*100
            df_result = pd.concat([df_result,x,y,z])

        df_result = df_result.drop(columns=['GOODS_NO'])

        df_result = df_result.reset_index().rename(columns = {'index':'ID'})
        now = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
        df_result['CHECK_TIME']=now
        df_result['CHECK_TIME']= pd.to_datetime(df_result['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')

        df_result['SCORE'] = df_result['SCORE'].astype(float)
        Write_Oracle().write_oracle(f'{TableList.BD_RISK_RESULT_WAREHOUSE_WH1.value}', df_result,org_code=self.org_code,alarm=None)
        
        # 整理预警明细数据，并写入数据库
        RISK_ALARM = df_result[df_result['SCORE'] != 0].groupby(['RISK_LABEL'], as_index=False)['ID'].count()
        RISK_ALARM = RISK_ALARM.rename(columns={'ID':'ALARM_NUMBER'})
        RISK_ALARM['ALARM_REASON'] = '发现' + RISK_ALARM['ALARM_NUMBER'].astype('str') + '起' + RISK_ALARM['RISK_LABEL'] + '事件'
        RISK_ALARM['CHECK_TIME'] = datetime.datetime.strptime(now, "%Y-%m-%d %H:%M:%S")
        RISK_ALARM['ORG_CODE'] = self.org_code
        RISK_ALARM['MODEL_CODE'] = 'WAREHOUSE'
        RISK_ALARM['CHILD_MODEL_CODE'] = 'WH2'
        RISK_ALARM['ID'] = range(len(RISK_ALARM))
        RISK_ALARM = RISK_ALARM[['ID','ORG_CODE','MODEL_CODE','CHILD_MODEL_CODE','ALARM_REASON','ALARM_NUMBER','CHECK_TIME']].copy()
        
        if RISK_ALARM.empty:
            print('没有异常情况')
        else:
            Write_Oracle().write_oracle('BD_RISK_ALARM_ITEM',RISK_ALARM, org_code = self.org_code, alarm = ['WAREHOUSE','WH1'])


    def run_model_wh1(self):
        try:
            self.model_wh1()
            exec_status = 1
        except:
            logger.exception('model execution error')
            exec_status = 0
        finally:
            Risk_logger(child_task_id=self.child_task_id, exec_status=exec_status).write_log()


if __name__ == '__main__':
#     child_task_id = sys.argv[1]
    # 读取传入的 child_task_id，仅适用于py调用,先备注掉
    child_task_id = '0001_0005'
    org_code, param_json, base_time = read_log_table(child_task_id)
    org_code = '91310115688774070B'
    WarehouseModelWh1(org_code, params=param_json, base_time = base_time, child_task_id=child_task_id).run_model_wh1()

2021-06-04 17:25:08.425 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 5.103s
2021-06-04 17:25:09.603 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 1.173s
2021-06-04 17:25:13.953 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.078s
2021-06-04 17:25:15.744 | INFO     | risk_models.config.write_config.write_func:write_oracle:179 - Processing... Writing 25792 rows into database
2021-06-04 17:25:19.593 | INFO     | risk_models.config.write_config.write_func:write_oracle:186 - Insert data into BD_RISK_RESULT_WAREHOUSE_WH1 successfully! Total write time spent 5.806s
2021-06-04 17:25:19.727 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.093s
2021-06-04 17:25:19.857 | INFO     | risk_models.config.write

In [1]:
import sys, os
from os import path
# sys.path.append('/root/bdrisk/risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *


class WarehouseModelWh1(object):
    def __init__(self, org_code, params, base_time, child_task_id):
        self.model_code = 'WAREHOUSE'
        self.child_model_code = 'WH1'
        self.child_task_id = child_task_id
        Risk_logger(child_task_id=self.child_task_id, exec_status=None)

        # 参数读取
        self.org_code = org_code
        # [正常，匹配不上报关单，匹配不上货物，贸易金额不符，法定数量不符，申报数量1不符，申报数量2不符，三种数量无法对应]
#         self.score_crit = json.loads(params)['score_crit']
        self.score_crit = [0,0,0,-1,-1,-1,-1,-1]


    def model_wh1(self):

        def thr_sup(num_x):
            if pd.isnull(num_x):
                return 0
            else:
                return (1+0.05)*num_x
            
        def thr_inf(num_x):
            if pd.isnull(num_x):
                return 0
            else:
                return (1-0.05)*num_x

        sql = f"""\
        SELECT * FROM {TableList.BD_RISK_DETAIL_WAREHOUSE_WH1_C.value} WHERE ISCURRENT = 1 AND ORG_CODE LIKE '{self.org_code}'
        """
        CUS_DATA = Read_Oracle().read_oracle(sql=sql, database='dbods')

        sql = f"""\
        SELECT * FROM {TableList.BD_RISK_DETAIL_WAREHOUSE_WH1_S.value} WHERE ISCURRENT = 1 AND ORG_CODE LIKE '{self.org_code}'
        """
        STOCK_DATA = Read_Oracle().read_oracle(sql=sql, database='dbods')

        entry_list = CUS_DATA[['ENTRY_NO','GOODS_NO']].copy()
        entry_list = entry_list.drop_duplicates()
        entry_list['conn_flag'] = 1

        entry_only = CUS_DATA[['ENTRY_NO']].copy()
        entry_only = entry_only.drop_duplicates()
        entry_only['conn_flag_e'] = 1

        df = pd.merge(STOCK_DATA,entry_list,how = 'left',left_on=['ENTRY_NO','ENTRY_GDS_SEQNO'],right_on=['ENTRY_NO','GOODS_NO'])
        df = pd.merge(df,entry_only,how = 'left',left_on=['ENTRY_NO'],right_on=['ENTRY_NO'])

        df_match = df[-pd.isnull(df['conn_flag_e'])].copy()
        df_result = df[pd.isnull(df['conn_flag_e'])].copy()
        df_result = df_result.drop(columns = ['ID','ISCURRENT','CHECK_TIME','conn_flag_e','conn_flag','LASTUPDATE'])
        df_match = df_match.drop(columns = ['ID','ISCURRENT','CHECK_TIME','conn_flag_e','LASTUPDATE'])
        df_result['RISK_LABEL'] = '匹配不到相应报关单'
        df_result['SCORE'] = self.score_crit[1]

        x = df_match[pd.isnull(df_match['conn_flag'])].copy()
        x = x.drop(columns=['conn_flag'])
        x['RISK_LABEL'] = '匹配不到相应货物'
        x['SCORE'] = self.score_crit[2]
        df_result = pd.concat([df_result,x])
        df_match = df_match[-pd.isnull(df_match['conn_flag'])].copy()
        df_match = df_match.drop(columns=['conn_flag'])

        if df_match.shape[0]!=0:
            CHECK_DATA = CUS_DATA[['ENTRY_NO','GOODS_NO','DECL_TOTAL']].copy()
            CHECK_DATA['trade_thr_sup'] = CHECK_DATA['DECL_TOTAL'].apply(thr_sup)
            CHECK_DATA['trade_thr_inf'] = CHECK_DATA['DECL_TOTAL'].apply(thr_inf)
            CHECK_DATA = CHECK_DATA.drop(columns=['DECL_TOTAL'])
            df_match = pd.merge(df_match,CHECK_DATA,how='inner',left_on=['ENTRY_NO','GOODS_NO'],right_on=['ENTRY_NO','GOODS_NO'])
            x = df_match[(df_match['TRADE_TOTAL']>df_match['trade_thr_sup'])|(df_match['TRADE_TOTAL']<df_match['trade_thr_inf'])].copy()
            match_count = df_match.shape[0]
            x = x.drop(columns=['trade_thr_sup','trade_thr_inf'])
            x['RISK_LABEL'] = '贸易金额超出阈值'
            x['SCORE'] = self.score_crit[3]/match_count*100
            df_match = df_match[(df_match['TRADE_TOTAL']<=df_match['trade_thr_sup'])&(df_match['TRADE_TOTAL']>=df_match['trade_thr_inf'])].copy()
            df_match = df_match.drop(columns=['trade_thr_sup','trade_thr_inf'])
            df_result = pd.concat([df_result,x])

        if df_match.shape[0]!=0:
            CHECK_DATA = CUS_DATA[['ENTRY_NO','GOODS_NO','G_UNIT','UNIT_1','UNIT_2','G_QTY','QTY_1','QTY_2']].copy()
            CHECK_DATA['qty_thr_sup_0'] = CHECK_DATA['G_QTY'].apply(thr_sup)
            CHECK_DATA['qty_thr_inf_0'] = CHECK_DATA['G_QTY'].apply(thr_inf)
            CHECK_DATA['qty_thr_sup_1'] = CHECK_DATA['QTY_1'].apply(thr_sup)
            CHECK_DATA['qty_thr_inf_1'] = CHECK_DATA['QTY_1'].apply(thr_inf)
            CHECK_DATA['qty_thr_sup_2'] = CHECK_DATA['QTY_2'].apply(thr_sup)
            CHECK_DATA['qty_thr_inf_2'] = CHECK_DATA['QTY_2'].apply(thr_inf)
            CHECK_DATA = CHECK_DATA.drop(columns=['G_QTY','QTY_1','QTY_2'])
            CHECK_DATA.columns = ['ENTRY_NO','GOODS_NO','UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2']
            CHECK_DATA['UNIT_CHECK_0'] = CHECK_DATA['UNIT_CHECK_0'].astype(str)
            CHECK_DATA['UNIT_CHECK_1'] = CHECK_DATA['UNIT_CHECK_1'].astype(str)
            CHECK_DATA['UNIT_CHECK_2'] = CHECK_DATA['UNIT_CHECK_2'].astype(str)

            df_match = pd.merge(df_match,CHECK_DATA,how='inner',left_on=['ENTRY_NO','GOODS_NO'],right_on=['ENTRY_NO','GOODS_NO'])

            x = df_match[df_match['G_UNIT']==df_match['UNIT_CHECK_0']].copy()
            df_match = df_match[df_match['G_UNIT']!=df_match['UNIT_CHECK_0']].copy()

            y = x[(x['QTY']>x['qty_thr_sup_0'])|(x['QTY']<x['qty_thr_inf_0'])].copy()
            x = x[(x['QTY']<=x['qty_thr_sup_0'])&(x['QTY']>=x['qty_thr_inf_0'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量超出阈值'
            y['SCORE'] = self.score_crit[4]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['G_UNIT']==df_match['UNIT_CHECK_1']].copy()
            df_match = df_match[df_match['G_UNIT']!=df_match['UNIT_CHECK_1']].copy()

            y = x[(x['QTY']>x['qty_thr_sup_1'])|(x['QTY']<x['qty_thr_inf_1'])].copy()
            x = x[(x['QTY']<=x['qty_thr_sup_1'])&(x['QTY']>=x['qty_thr_inf_1'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量超出阈值'
            y['SCORE'] = self.score_crit[4]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['G_UNIT']==df_match['UNIT_CHECK_2']].copy()
            df_match = df_match[df_match['G_UNIT']!=df_match['UNIT_CHECK_2']].copy()

            y = x[(x['QTY']>x['qty_thr_sup_2'])|(x['QTY']<x['qty_thr_inf_2'])].copy()
            x = x[(x['QTY']<=x['qty_thr_sup_2'])&(x['QTY']>=x['qty_thr_inf_2'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量超出阈值'
            y['SCORE'] = self.score_crit[4]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['UNIT_1']==df_match['UNIT_CHECK_0']].copy()
            df_match = df_match[df_match['UNIT_1']!=df_match['UNIT_CHECK_0']].copy()

            y = x[(x['QTY_1']>x['qty_thr_sup_0'])|(x['QTY_1']<x['qty_thr_inf_0'])].copy()
            x = x[(x['QTY_1']<=x['qty_thr_sup_0'])&(x['QTY_1']>=x['qty_thr_inf_0'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量匹配不上，法定数量1超出阈值'
            y['SCORE'] = self.score_crit[5]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['UNIT_1']==df_match['UNIT_CHECK_1']].copy()
            df_match = df_match[df_match['UNIT_1']!=df_match['UNIT_CHECK_1']].copy()

            y = x[(x['QTY_1']>x['qty_thr_sup_1'])|(x['QTY_1']<x['qty_thr_inf_1'])].copy()
            x = x[(x['QTY_1']<=x['qty_thr_sup_1'])&(x['QTY_1']>=x['qty_thr_inf_1'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量匹配不上，法定数量1超出阈值'
            y['SCORE'] = self.score_crit[5]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['UNIT_1']==df_match['UNIT_CHECK_2']].copy()
            df_match = df_match[df_match['UNIT_1']!=df_match['UNIT_CHECK_2']].copy()

            y = x[(x['QTY_1']>x['qty_thr_sup_2'])|(x['QTY_1']<x['qty_thr_inf_2'])].copy()
            x = x[(x['QTY_1']<=x['qty_thr_sup_2'])&(x['QTY_1']>=x['qty_thr_inf_2'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '申报数量匹配不上，法定数量1超出阈值'
            y['SCORE'] = self.score_crit[5]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['UNIT_2']==df_match['UNIT_CHECK_0']].copy()
            df_match = df_match[df_match['UNIT_2']!=df_match['UNIT_CHECK_0']].copy()

            y = x[(x['QTY_2']>x['qty_thr_sup_0'])|(x['QTY_2']<x['qty_thr_inf_0'])].copy()
            x = x[(x['QTY_2']<=x['qty_thr_sup_0'])&(x['QTY_2']>=x['qty_thr_inf_0'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '仅法定数量2可匹配，法定数量2超出阈值'
            y['SCORE'] = self.score_crit[6]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['UNIT_2']==df_match['UNIT_CHECK_1']].copy()
            df_match = df_match[df_match['UNIT_2']!=df_match['UNIT_CHECK_1']].copy()

            y = x[(x['QTY_2']>x['qty_thr_sup_1'])|(x['QTY_2']<x['qty_thr_inf_1'])].copy()
            x = x[(x['QTY_2']<=x['qty_thr_sup_1'])&(x['QTY_2']>=x['qty_thr_inf_1'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '仅法定数量2可匹配，法定数量2超出阈值'
            y['SCORE'] = self.score_crit[6]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            df_result = pd.concat([df_result,x,y])

        if df_match.shape[0]!=0:
            x = df_match[df_match['UNIT_2']==df_match['UNIT_CHECK_2']].copy()
            z = df_match[df_match['UNIT_2']!=df_match['UNIT_CHECK_2']].copy()

            y = x[(x['QTY_2']>x['qty_thr_sup_2'])|(x['QTY_2']<x['qty_thr_inf_2'])].copy()
            x = x[(x['QTY_2']<=x['qty_thr_sup_2'])&(x['QTY_2']>=x['qty_thr_inf_2'])].copy()
            y = y.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            y['RISK_LABEL'] = '仅法定数量2可匹配，法定数量2超出阈值'
            y['SCORE'] = self.score_crit[6]/match_count*100
            x = x.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            x['RISK_LABEL'] = '数据正常'
            x['SCORE'] = self.score_crit[0]
            z = z.drop(columns=['UNIT_CHECK_0','UNIT_CHECK_1','UNIT_CHECK_2','qty_thr_sup_0','qty_thr_inf_0','qty_thr_sup_1','qty_thr_inf_1','qty_thr_sup_2','qty_thr_inf_2'])
            z['RISK_LABEL'] = '申报数量、法定数量1与法定数量2均匹配失败'
            z['SCORE'] = self.score_crit[7]/match_count*100
            df_result = pd.concat([df_result,x,y,z])

        df_result = df_result.drop(columns=['GOODS_NO'])

        df_result = df_result.reset_index().rename(columns = {'index':'ID'})
        now = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
        df_result['CHECK_TIME']=now
        df_result['CHECK_TIME']= pd.to_datetime(df_result['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')
        print(df_result['SCORE'].sum())

        df_result['SCORE'] = df_result['SCORE'].astype(float)
        Write_Oracle().write_oracle(f'{TableList.BD_RISK_RESULT_WAREHOUSE_WH1.value}', df_result,org_code=self.org_code,alarm=None)
        
        # 整理预警明细数据，并写入数据库
        RISK_ALARM = df_result[df_result['SCORE'] != 0].groupby(['RISK_LABEL'], as_index=False)['ID'].count()
        RISK_ALARM = RISK_ALARM.rename(columns={'ID':'ALARM_NUMBER'})
        RISK_ALARM['ALARM_REASON'] = '发现' + RISK_ALARM['ALARM_NUMBER'].astype('str') + '起' + RISK_ALARM['RISK_LABEL'] + '事件'
        RISK_ALARM['CHECK_TIME'] = datetime.datetime.strptime(now, "%Y-%m-%d %H:%M:%S")
        RISK_ALARM['ORG_CODE'] = self.org_code
        RISK_ALARM['MODEL_CODE'] = 'WAREHOUSE'
        RISK_ALARM['CHILD_MODEL_CODE'] = 'WH1'
        RISK_ALARM['ID'] = range(len(RISK_ALARM))
        RISK_ALARM = RISK_ALARM[['ID','ORG_CODE','MODEL_CODE','CHILD_MODEL_CODE','ALARM_REASON','ALARM_NUMBER','CHECK_TIME']].copy()
        
        if RISK_ALARM.empty:
            print('没有异常情况')
        else:
            Write_Oracle().write_oracle('BD_RISK_ALARM_ITEM',RISK_ALARM, org_code = self.org_code, alarm = ['WAREHOUSE','WH1'])


    def run_model_wh1(self):
        try:
            self.model_wh1()
            exec_status = 1
        except:
            logger.exception('model execution error')
            exec_status = 0
        finally:
            Risk_logger(child_task_id=self.child_task_id, exec_status=exec_status).write_log()


if __name__ == '__main__':
#     child_task_id = sys.argv[1]
    # 读取传入的 child_task_id，仅适用于py调用,先备注掉
    child_task_id = '0001_0005'
    org_code, param_json, base_time = read_log_table(child_task_id)
    org_code = '91310115688774070B'
    WarehouseModelWh1(org_code, params=param_json, base_time = base_time, child_task_id=child_task_id).run_model_wh1()

2021-06-08 09:21:30.394 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 2.046s
2021-06-08 09:21:37.143 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 6.702s
2021-06-08 09:21:38.946 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 1.799s


-1.7777777777777777


2021-06-08 09:21:43.588 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.150s
2021-06-08 09:21:51.155 | INFO     | risk_models.config.write_config.write_func:write_oracle:179 - Processing... Writing 25792 rows into database
2021-06-08 09:22:04.434 | INFO     | risk_models.config.write_config.write_func:write_oracle:186 - Insert data into BD_RISK_RESULT_WAREHOUSE_WH1 successfully! Total write time spent 21.09s
2021-06-08 09:22:04.759 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.229s
2021-06-08 09:22:04.905 | INFO     | risk_models.config.write_config.write_func:write_oracle:179 - Processing... Writing 3 rows into database
2021-06-08 09:22:04.945 | INFO     | risk_models.config.write_config.write_func:write_oracle:186 - Insert data into BD_RISK_ALARM_ITEM successfully! Total write time spent 0.419s
2021-06-08 09:22:05.186 | INFO     | risk_